In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import gc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
warnings.simplefilter('ignore')

In [3]:
# dataの読み込み
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
submit_df = pd.read_csv('./submit_sample.csv',header=None)

FileNotFoundError: ignored

In [4]:
# google colab
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/signate/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/signate/test.csv")
submit_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/signate/submit_sample.csv", header=None)

In [5]:
# データの量の確認
train_df.shape,test_df.shape,submit_df.shape

((27100, 18), (18050, 17), (18050, 2))

In [6]:
# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['y']=-999

In [7]:
# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

0

In [8]:
# カテゴリカラムの前処理
categorical_features = ['job', 'marital', 'education','default','housing','loan','contact','month','poutcome']
for col in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(all_df[col])
    lbl.transform(all_df[col])
    all_df[col]=lbl.transform(all_df[col])

In [9]:
#fe = ['age', 'marital', 'education', 'housing', 'loan', 'contact' ,'month', 'campaign', 'previous', 'poutcome']
#all_df['log_bal'] = np.log(all_df['balance'])
#all_df['bal_div_age'] = all_df['balance'] / all_df['age']
#all_df['bal_mul_age'] = all_df['balance'] * all_df['age']
#all_df['bal_div_age_p3'] = np.power(all_df['bal_div_age'], 3)
#sall_df['campaign_p2'] = np.power(all_df['campaign'], 2)
all_df['housing_and_loan'] = all_df['housing'] & all_df['loan']
all_df['month_q'] = all_df['month'] / 3



#all_df.drop(['job', 'balance', 'day', 'duration', 'pdays'], axis=1)


In [44]:
def link_mean_job():
    mean_job_df = all_df.groupby(['job'], as_index=False).mean()
    result_feature_map = np.zeros(len(all_df))
    for i, row in enumerate(all_df.itertuples()):
        job_index = row.job
        result_feature_map[i] = np.array(row.balance) - np.array(mean_job_df['balance'][job_index])

    all_df['sub_job_mean'] = result_feature_map

In [43]:
mean_job_df = all_df.groupby(['job'], as_index=False).mean()
print("###################")
print(mean_job_df['balance'])
print('-------------------')
print(mean_job_df['balance'][11])


###################
0     47253.409440
1     47222.778453
2     48680.265393
3     46480.264800
4     47230.890792
5     47923.479831
6     48492.076190
7     47143.182968
8     45672.653516
9     47898.966905
10    48348.106432
11    10997.000000
Name: balance, dtype: float64
-------------------
10997.0


In [45]:
link_mean_job()

In [49]:
# check 'sub_job_mean'
print(np.sum(all_df['sub_job_mean']))

5.587935447692871e-09


In [50]:
# 訓練データ、テストデータの分割
train_df = all_df[all_df['y']!=-999]
test_df = all_df[all_df['y']==-999]

In [51]:
y_train = train_df['y']
X_train = train_df.drop(['y','id'], axis=1)
X_test = test_df.drop(['y','id'], axis=1)

In [52]:
from sklearn.model_selection import KFold
from sklearn import metrics


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)


params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)
    
    # eval
    #acc = accuracy_score(y_val, y_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_val, oof_train[valid_index])
    auc = metrics.auc(fpr, tpr)
    print('auc ', auc)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.228789	valid_1's binary_logloss: 0.215759
[20]	training's binary_logloss: 0.210902	valid_1's binary_logloss: 0.203479
[30]	training's binary_logloss: 0.200044	valid_1's binary_logloss: 0.19729
[40]	training's binary_logloss: 0.191335	valid_1's binary_logloss: 0.193117
[50]	training's binary_logloss: 0.184106	valid_1's binary_logloss: 0.190464
[60]	training's binary_logloss: 0.178234	valid_1's binary_logloss: 0.188763
[70]	training's binary_logloss: 0.172832	valid_1's binary_logloss: 0.187823
[80]	training's binary_logloss: 0.168216	valid_1's binary_logloss: 0.187372
[90]	training's binary_logloss: 0.163916	valid_1's binary_logloss: 0.186785
[100]	training's binary_logloss: 0.15969	valid_1's binary_logloss: 0.18687
Early stopping, best iteration is:
[92]	training's binary_logloss: 0.163079	valid_1's binary_logloss: 0.186683
auc  0.8646391694556785
Training until validation scores don't improv

In [53]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.18668266497212485, 0.20888507001477283, 0.20746194688662276, 0.19999751996625967, 0.20260285633098535]
0.2011260116341531


In [54]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.9307749077490775

In [55]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, oof_train)
metrics.auc(fpr, tpr)

# baseline is 0.8512576188085433
# 0.8517739935623644

0.8507803253137263

In [56]:
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_train, y_pred_oof)

In [57]:
cm

array([[24782,   206],
       [ 1670,   442]])

## Reference
- CV
    - https://www.kaggle.com/sishihara/upura-kaggle-tutorial-06-validation
- Metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html#sklearn.metrics.auc